In [ ]:
#| default_exp tabulator 

# Exploring your remote data with tabulator

> Another try using panel 

Ok, let's collect the contents of our falnama-project. 

In [ ]:
from fairdatanow import RemoteData2
import os 

In [ ]:
configuration = {
    'url': "https://laboppad.nl/falnama-project", 
    'user':    os.getenv('NC_AUTH_USER'),
    'password': os.getenv('NC_AUTH_PASS')
}

In [ ]:
remote_data = RemoteData2(configuration)

In [ ]:
file_table = remote_data.listdir('falnama-project')

file_table 

Please wait while scanning all file paths in remote folder...


Column
    [0] Row
        [0] TextInput(name='Search filter', value='xray')
        [1] Checkbox(name='show directories')
        [2] MultiChoice(name='filter extensions', options=['', '.jpg', '.json', ...])
    [1] Tabulator(height=350, show_index=False, value=              ...)
    [2] Str(str)

To be continued...

Here is an embedded iframe: 

<iframe width="780" height="500" src="iframes/file_table.html" title="Webpage example"></iframe>

And a [link](iframes/file_table.html)

## FUNCTIONS 

In [ ]:
#| export 

import nc_py_api 
from nc_py_api import Nextcloud 
import panel as pn
import param 
import humanize
import pandas as pd
import os 
import re
import html 

import ipynb_path

In [ ]:
#| export 

pn.extension('tabulator')

def _node_to_dataframe2(fsnode): 
    '''Convert `fsnode` object to polars a single row polars dataframe.'''

    df = pd.DataFrame({'path': [fsnode.user_path], 'size': [fsnode.info.size], 'mimetype': [fsnode.info.mimetype], 'modified': [fsnode.info.last_modified], 
                   'isdir': [fsnode.is_dir], 'ext': [os.path.splitext(fsnode.user_path)[1]]})

    return df 

def to_iframe(panel_layout, html_file, height=500): 
    '''Embed interactive `panel_layout` object  as full HTML page in an iframe in a panel HTML pane. 
    
    In this way it should be possible to preserve rich interactive visualizations directly in web pages.

    See: https://panel.holoviz.org/reference/panes/HTML.html#html-documents  
    '''
    # Create iframes subfolder 
    notebooks_dir = os.path.dirname(ipynb_path.get())
    iframes_dir = os.path.join(notebooks_dir, 'iframes')
    os.makedirs(iframes_dir, exist_ok=True)
    
    # Save the plot as html file to iframes subfolder 
    html_path = os.path.join(iframes_dir, html_file)
    panel_layout.save(html_path)
    
    markdown_string = '```{=html}\n' + f'<iframe width="780" height="500" src="iframes/{html_file}" title="Webpage example"></iframe>\n' + '```\n'

    return markdown_string

    

class RemoteData2(object): 
    
    # See: https://help.nextcloud.com/t/using-nc-py-api-i-cant-download-any-file-due-to-ssl-certificte-verify-failed/194019 
    nc_py_api.options.NPA_NC_CERT = False 

    def __init__(self, configuration): 
        '''Recursively scan the contents of a remote webdav server as specified by `configuration`. 
        '''

        # parse configuration 
        m = re.match('(^https://[^/]+/)(.*)', configuration['url'])
        nextcloud_url, self.cache_dir = m.groups()
        nc_auth_user = configuration['user']
        nc_auth_pass = configuration['password'] 
               
        # Instantiate Nextcloud client 
        self.nc = Nextcloud(nextcloud_url=nextcloud_url, nc_auth_user=nc_auth_user, nc_auth_pass=nc_auth_pass) 
        

    def listdir(self, subdir=None, search_regex='', searchBuilder={}): 
        '''Create interactive file table for remote subdirectory `subdir`. 

        If subdir is not specified the complete project directory is scanned. 
        '''

        if subdir is None: 
            subdir = self.cache_dir 

        print(f'Please wait while scanning all file paths in remote folder...') 
            
        # query webdav server to obtain file listing 
        fs_nodes_list = self.nc.files.listdir(subdir, depth=-1, exclude_self=False) 
        
        n_paths = len(fs_nodes_list)

        # initialize polars dataframe with first row to fix schema 
        self.df = _node_to_dataframe2(fs_nodes_list[0]) 
        
        #sum the sizes to find the total storage space
        total_size_bytes = self.df['size'].sum()
        total_size = humanize.naturalsize(total_size_bytes, True)

        
        for fsnode in fs_nodes_list[1:]: 
            self.df = pd.concat([self.df, _node_to_dataframe2(fsnode)], ignore_index=True) 

        self.df.reset_index()
        
        # panel components   
        self.search_filter = pn.widgets.TextInput(name='Search filter', value='xray') 
        self.isdir_cb = pn.widgets.Checkbox(name='show directories')
        self.type_select = pn.widgets.MultiChoice(name='filter extensions',options=self.df['ext'].unique().tolist())
        
        self.file_table = pn.widgets.Tabulator(self.df, height=350, pagination=None, show_index=False)
        
        self.row_counter = pn.pane.Str(f"Showing {len(self.df)} out of {len(self.df)} rows")


        # update file table and row counter to search filter 
        self.file_table.add_filter(pn.bind(self._contains_filter, pattern=self.search_filter, column='path'))
        self.file_table.add_filter(pn.bind(self._show_directories, column='isdir'))
        self.file_table.add_filter(self.type_select, 'ext')

        # create panel layout
        self.top_row = pn.Row(self.search_filter, self.isdir_cb, self.type_select)
        self.layout = pn.Column(self.top_row, self.file_table, self.row_counter)
        

        return self.layout 


    
    def _contains_filter(self, df, pattern, column): 
        '''String contains `pattern` filter function on 'column` of dataframe `df`. '''
        
        if not pattern:
            self._update_row_counter(df)
            return df 
            
        filtered_df = df[df[column].str.contains(pattern)]
        
        self._update_row_counter(filtered_df)
        
        return filtered_df

    def _show_directories(self, df, column):
        '''When the show directories checkbox is True then show everything.
        If False only show files.'''

        if self.isdir_cb.value:
            self._update_row_counter(df)
            return df

        filtered_df = df[df[column] == False]

        self._update_row_counter(filtered_df)
        
        return filtered_df

    def _update_row_counter(self, filtered_df):
        '''Updates the value of the row counter'''
        self.row_counter.object = f"Showing {len(filtered_df)} out of {len(self.df)} rows"
